### 1. Setup inicial do Spark e Postgres

In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import broadcast

from functools import reduce
import psycopg2

import sys
sys.path.append("../libs")
from utils import silenciar_warnings, pgconfig_init, pg_conn, pg_executar_sql, spark_init, spark_read_jdbc

# Silenciar warnings/logs
silenciar_warnings()

#### 1.1 Spark

In [2]:
# Inicializar Spark
spark = spark_init("IBGE_Gold")

:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/hgirardi/.ivy2.5.2/cache
The jars for the packages stored in: /Users/hgirardi/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7a1b5eaf-264c-41a4-8afd-84d6a25138e4;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 45ms :: artifacts dl 3ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------

✅ Spark inicializado: IBGE_Gold [local[*]]


#### 1.2 Postgres

In [3]:
# initizar configuração do PostgreSQL
pgConfig = pgconfig_init()

# criar schema silver no PostgreSQL, se não existir
with pg_conn(pgConfig) as conn:
    pg_executar_sql(conn, "CREATE SCHEMA IF NOT EXISTS gold;")

✅ Configuração do Postgres carregada.
== Executando SQL no Postgres ==
✅ SQL executado!
== Processo finalizado ==


### 2. Carregar Silver

In [ ]:
# carregar tabela silver.covid
df_silver = spark_read_jdbc(spark, "silver.covid", pgConfig)

df_silver.createOrReplaceTempView("silver_covid")
spark.sql("CACHE TABLE silver_covid")  # acelera as próximas queries
_ = spark.table("silver_covid").limit(1).count()

DataFrame[]

In [7]:
_ = spark.table("silver_covid").limit(1).count()